In [1]:
import psycopg2
from dotenv import load_dotenv
import os
import pandas as pd
import warnings
import seaborn

ModuleNotFoundError: No module named 'psycopg2'

In [ ]:
repo = os.getenv("REPO","apache/airflow")
dbname = os.getenv("PG_DB", "app")
user = os.getenv("PG_USER", "postgres")
password = os.getenv("PG_PW", "example")
dbhost = os.environ.get("PG_HOST","localhost")

In [ ]:
def query_to_dataframe(sqlquery): 
    with psycopg2.connect(dbname=dbname, user=user, password=password, host=dbhost) as conn:
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            data = pd.read_sql(sqlquery,conn)
    return data

## Top 5 committers in the last 6 months

In [ ]:
top5query = "select distinct author, count(id) from app.ft_commits where datetime > (CURRENT_DATE - interval '6 month') group by author order by count desc limit 5"

query_to_dataframe(top5query)

## Top 5 longest commit streaks

In [ ]:
streakquery = """select 
author, 
min(commitdate) streak_start, 
max(commitdate) streak_end, 
count(commitdate) streak_length 
from (
    select 
    author, 
    commitdate, 
    commitdate::date - rownum::int as category
    from (
        select distinct 
            author, 
            date(datetime) as commitdate, 
            dense_rank() over (partition by author order by date(datetime)) as rownum
            from app.ft_commits
            where datetime > (current_date) - interval '6 month'
            order by author, commitdate
        ) a
    )  b
group by b.author, b.category 
order by streak_length desc 
limit 5"""

query_to_dataframe(streakquery)

## Heatmap of commits by day of week and time of day

In [ ]:
sql = "select datetime from app.ft_commits where datetime > (current_date - interval '6 month')"
data = query_to_dataframe(sql)
data["Day of Week"] = data["datetime"].dt.day_name()

data['Day of Week'] = data['Day of Week'].astype('category')

data["Time of Day"] = data["datetime"].dt.hour.map(lambda x: "00-03" if x < 3 
                                                   else "03-06" if x < 6 
                                                   else "06-09" if x < 9
                                                  else "09-12" if x < 12
                                                  else "12-15" if x < 15
                                                  else "15-18" if x < 18
                                                  else "18-21" if x < 21
                                                  else "21-00")
pivoted = data.groupby(["Day of Week","Time of Day"],observed=False).count().reset_index().pivot(index="Day of Week", columns="Time of Day", values="datetime")

# Currently the days of week are being sorted alphabetically, reorder them
dayorder = ["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"]
pivoted.index = pivoted.index.reorder_categories(dayorder, ordered=True)
pivoted.sort_index(axis ="index",inplace=True)

seaborn.heatmap(pivoted, cmap="crest")